In [168]:
import numpy as np
import torch

np.random.seed(2)

T = 5
L = 30
N = 6000

x = np.empty((N, L), 'int64')
x[:] = np.array(range(L)) + np.random.randint(-10 * T, 10 * T, N).reshape(N, 1)
data = np.sin(x / 1.0 / T).astype('float64')
data = np.expand_dims(data, axis=2)
print(data[0])

torch.save(data, open('targets.pt', 'wb'))

zero_array = np.zeros_like(data)
data = np.concatenate((reshaped_array, zero_array), axis=2)
data = np.concatenate((data, zero_array), axis=2)
print(data[0])

torch.save(data, open('traindata.pt', 'wb'))

[[-0.90929743]
 [-0.97384763]
 [-0.9995736 ]
 [-0.98544973]
 [-0.93203909]
 [-0.84147098]
 [-0.71735609]
 [-0.56464247]
 [-0.38941834]
 [-0.19866933]
 [ 0.        ]
 [ 0.19866933]
 [ 0.38941834]
 [ 0.56464247]
 [ 0.71735609]
 [ 0.84147098]
 [ 0.93203909]
 [ 0.98544973]
 [ 0.9995736 ]
 [ 0.97384763]
 [ 0.90929743]
 [ 0.8084964 ]
 [ 0.67546318]
 [ 0.51550137]
 [ 0.33498815]
 [ 0.14112001]
 [-0.05837414]
 [-0.2555411 ]
 [-0.44252044]
 [-0.61185789]]
[[-0.90929743  0.          0.        ]
 [-0.97384763  0.          0.        ]
 [-0.9995736   0.          0.        ]
 [-0.98544973  0.          0.        ]
 [-0.93203909  0.          0.        ]
 [-0.84147098  0.          0.        ]
 [-0.71735609  0.          0.        ]
 [-0.56464247  0.          0.        ]
 [-0.38941834  0.          0.        ]
 [-0.19866933  0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.19866933  0.          0.        ]
 [ 0.38941834  0.          0.        ]
 [ 0.56464247  0.          0.        ]
 [ 

In [5]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

class Sequence(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CustomLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm1 = nn.LSTMCell(3, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 1)

    def forward(self, input, future = 0):
        outputs = []
        batch_size = input.size(0)
        seq_len = input.size(1)
        h_t = torch.zeros(batch_size, 51, dtype=torch.double)
        c_t = torch.zeros(batch_size, 51, dtype=torch.double)
        h_t2 = torch.zeros(batch_size, 51, dtype=torch.double)
        c_t2 = torch.zeros(batch_size, 51, dtype=torch.double)

        for t in range(seq_len):
            input_t = input[:, t, :]
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs.append(output.unsqueeze(1))
        outputs = torch.cat(outputs, dim=1)
        return outputs


# load data and make training set
data = torch.load('traindata.pt')
targets = torch.load('targets.pt')
input = torch.from_numpy(data[3:, :-1])
target = torch.from_numpy(targets[3:, 1:])
test_input = torch.from_numpy(data[:3, :-1])
test_target = torch.from_numpy(targets[:3, 1:])

# build the model
seq = Sequence()
seq.double()
criterion = nn.MSELoss()
optimizer = optim.LBFGS(seq.parameters(), lr=0.8)
#begin to train
print("start")
for i in range(15):
    print('STEP: ', i)
    def closure():
        optimizer.zero_grad()
#         print(input.size())
#         print(input)
        out = seq(input)
        loss = criterion(out, target)
        print('loss:', loss.item())
        loss.backward()
        return loss
    optimizer.step(closure)
    
    T = 5
    L = 60
    N = 1

    x = np.empty((N, L), 'int64')
    x[:] = np.array(range(L)) + np.random.randint(-10 * T, 10 * T, N).reshape(N, 1)
    data = np.sin(x / 1.0 / T).astype('float64')
    reshaped_array = np.expand_dims(data, axis=2)
    zero_array = np.zeros_like(reshaped_array)
    data = np.concatenate((reshaped_array, zero_array), axis=2)
    data = np.concatenate((data, zero_array), axis=2)
    data = torch.from_numpy(data[:, :])
        
    with torch.no_grad():
       
        future = 30
        predictions = []

        for day in range(future):
            # Predict one step ahead
            print(current_input.size())
            current_input = data[:,0+day:30+day]
            pred = seq(current_input)
            item = pred[:, -1].item()
            predictions.append(item)
    array1 = data.numpy()[0]
    array2 = predictions
    x1 = np.arange(len(array1))
    x2 = np.arange(len(array2)) + 30  # Offset the x-coordinates by 30 for the second array
    plt.figure(figsize=(10, 5))
    plt.plot(x1, array1, label='Actual', color='blue')
    plt.plot(x2, array2, label='Prediction', color='red')
    plt.savefig('predict%d.pdf'%i)
    plt.close()

start
STEP:  0
loss: 0.5491570009367831
loss: 0.5317029033488023
loss: 0.4965737247373802
loss: 0.48867894168703324
loss: 0.47142727743560064
loss: 0.4170638941476861
loss: 0.837451417594814
loss: 0.27092224626776984
loss: 0.2018124170776288
loss: 0.18414747183634605
loss: 0.164237725129691
loss: 0.1548864459880794
loss: 0.08375266315044962
loss: 0.057927064729938506
loss: 0.0359790471044725
loss: 0.026921969014545206
loss: 0.02335178727445269
loss: 0.02239831127499241
loss: 0.021901668200374324
loss: 0.02071382615087814
torch.Size([1, 30, 3])
torch.Size([1, 30, 3])
torch.Size([1, 30, 3])
torch.Size([1, 30, 3])
torch.Size([1, 30, 3])
torch.Size([1, 30, 3])
torch.Size([1, 30, 3])
torch.Size([1, 30, 3])
torch.Size([1, 30, 3])
torch.Size([1, 30, 3])
torch.Size([1, 30, 3])
torch.Size([1, 30, 3])
torch.Size([1, 30, 3])
torch.Size([1, 30, 3])
torch.Size([1, 30, 3])
torch.Size([1, 30, 3])
torch.Size([1, 30, 3])
torch.Size([1, 30, 3])
torch.Size([1, 30, 3])
torch.Size([1, 30, 3])
torch.Size([1

KeyboardInterrupt: 

In [100]:
# Random data creation
data = np.random.rand(5, 3)  # Assuming data has 5 rows and 3 columns

# Converting numpy array to PyTorch tensor
test_input = torch.from_numpy(data[:3, :-1])
test_target = torch.from_numpy(data[:3, 1:])

# Printing the results
print(data)
print("test_input:")
print(test_input)
print("test_target:")
print(test_target)

[[0.5488135  0.71518937 0.60276338]
 [0.54488318 0.4236548  0.64589411]
 [0.43758721 0.891773   0.96366276]
 [0.38344152 0.79172504 0.52889492]
 [0.56804456 0.92559664 0.07103606]]
test_input:
tensor([[0.5488, 0.7152],
        [0.5449, 0.4237],
        [0.4376, 0.8918]], dtype=torch.float64)
test_target:
tensor([[0.7152, 0.6028],
        [0.4237, 0.6459],
        [0.8918, 0.9637]], dtype=torch.float64)


In [156]:
T = 5
L = 60
N = 1

x = np.empty((N, L), 'int64')
x[:] = np.array(range(L)) + np.random.randint(-10 * T, 10 * T, N).reshape(N, 1)
data = np.sin(x / 1.0 / T).astype('float64')


reshaped_array = np.expand_dims(data, axis=2)
zero_array = np.zeros_like(reshaped_array)
data = np.concatenate((reshaped_array, zero_array), axis=2)
data = np.concatenate((data, zero_array), axis=2)
data = torch.from_numpy(data[:, :])
# print(data)

with torch.no_grad():
    future = 30
    predictions = []

    for day in range(future):
        # Predict one step ahead
        current_input = data[:,0+day:30+day]
        print(current_input)
#         pred = seq(current_input, future=1)
#         item = pred[:, -1].item()
#         predictions.append(item)  # Collect the last prediction (next day)
#         print(item)

#     print(predictions)
#     predictions = torch.cat(predictions, dim=1)  # Concatenate predictions
#     loss = criterion(predictions[:, :-future], test_target)
#     print('test loss:', loss.item())
#     y = predictions.detach().numpy()

array1 = data.numpy()[0]
array2 = predictions
x1 = np.arange(len(array1))
x2 = np.arange(len(array2)) + 31  # Offset the x-coordinates by 30 for the second array
plt.figure(figsize=(10, 5))
plt.plot(x1, array1, label='Actual', color='blue')
plt.plot(x2, array2, label='Prediction', color='red')

# Add title and labels
plt.title('Plot with X Offset')
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()
plt.grid(True)

# Display the plot
plt.savefig('hm.pdf')
plt.close()

tensor([[[-0.2555,  0.0000,  0.0000],
         [-0.4425,  0.0000,  0.0000],
         [-0.6119,  0.0000,  0.0000],
         [-0.7568,  0.0000,  0.0000],
         [-0.8716,  0.0000,  0.0000],
         [-0.9516,  0.0000,  0.0000],
         [-0.9937,  0.0000,  0.0000],
         [-0.9962,  0.0000,  0.0000],
         [-0.9589,  0.0000,  0.0000],
         [-0.8835,  0.0000,  0.0000],
         [-0.7728,  0.0000,  0.0000],
         [-0.6313,  0.0000,  0.0000],
         [-0.4646,  0.0000,  0.0000],
         [-0.2794,  0.0000,  0.0000],
         [-0.0831,  0.0000,  0.0000],
         [ 0.1165,  0.0000,  0.0000],
         [ 0.3115,  0.0000,  0.0000],
         [ 0.4941,  0.0000,  0.0000],
         [ 0.6570,  0.0000,  0.0000],
         [ 0.7937,  0.0000,  0.0000],
         [ 0.8987,  0.0000,  0.0000],
         [ 0.9679,  0.0000,  0.0000],
         [ 0.9985,  0.0000,  0.0000],
         [ 0.9894,  0.0000,  0.0000],
         [ 0.9407,  0.0000,  0.0000],
         [ 0.8546,  0.0000,  0.0000],
         [ 0

In [4]:
print(input.size())
print(target.size())

torch.Size([5997, 29, 3])
torch.Size([5997, 29, 1])
